In [ ]:
import pandas as pd
import numpy as np 
from pygam import LogisticGAM
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
import joblib

# Load data from ../Dataset

In [ ]:
df = pd.read_csv('../Dataset/df_processed.csv')

In [ ]:
df.head()

# Split data

In [ ]:
# Create label and features
X = df.drop('Grade', axis=1)
y = df['Grade']

In [ ]:
# Create train, test, and validation data
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Train a GAM classifier